### # Databricks notebook source

In [ ]:
dbutils.fs.ls("/Volumes/workspace/default/arquivos-aula/Anac/V_OCORRENCIA_AMPLA.json")

df = spark.read.json("/Volumes/workspace/default/arquivos-aula/Anac/V_OCORRENCIA_AMPLA.json")
display(df)

### # Identificar se existe mais de uma forma de escrita utilizando o 'distinct()'


In [ ]:
df.select("Fase_da_Operacao").distinct().show()

### # Ajustando letras maiúsculas e minúsculas para deixar a tabela com padrão de escrita


In [ ]:
# Deixando tudo em letra maiúscula:

from pyspark.sql.functions import upper, lower, initcap # Utilizar 'upper' para tudo maiúsculo, 'lower' para tudo minúsculo, e 'initcap' para deixar a primeira letra maiúscula (mesmo em palavras compostas ou frases. Ex: Primeira Letra Maiúscula)

display(df.select(upper("Fase_da_Operacao")).distinct()) # Upper

display(df.select(lower("Fase_da_Operacao")).distinct()) # Lower

display(df.select(initcap("Fase_da_Operacao")).distinct()) # Initcap

### # Para garantir que não haverá erro na busca dos dados por conta do case sensitive (diferença entre letras maiúsculas e minúsculas) é só deixar tudo em maiúsculo ou minúsculo e fazer a busca


In [ ]:
# Exemplo de busca com todas as letras do resultado da coluna (todas as letras das células) em maiúsculo
display(df.filter(upper(df.Fase_da_Operacao) == 'DECOLAGEM')) # Retorna todos os dados cuja fase da operação é 'Decolagem', sem distinção entre maiúsculas e minúsculas, porque se eu pesquisasse com 'D' maiúsculo poderia haver alguma célula preenchida com 'D' minúsculo e então não iria aparecer na pesquisa.

## Salva em um novo Data Frame

novo_df = df.filter(upper(df.Fase_da_Operacao) == 'DECOLAGEM')
display(novo_df)

## Resolvendo erros de digitação (espaços excessivos)

# Criando um Data Frame com dados fictícios para amostra de como corrigir os erros de 'espaço'

Dados = [(" São José dos Pinhais ", "PR", 100), # Espaços a mais de propósito
         ("São  Paulo", "SP", 20),
         ("Rio de Janeiro ", "RJ", 20),
         (" Brasília", "DF", 303030),
         ("Belo Horizonte", "MG", 252525)]

Colunas = ["Cidade", "UF", "Número"]

dataFrame = spark.createDataFrame(Dados, Colunas)

display(dataFrame)

### # Para corrigir o "erro" acima, é necessário importar as funções rtrim, ltrim e trim do sql.functions


In [ ]:
from pyspark.sql.functions import * # Ou importa nome por nome, ltrim, rtrim e trim

# trim: Remove os espaços em branco do início e do final de uma coluna
# ltrim: Remove os espaços em branco do início de uma coluna (esquerda)
# rtrim: Remove os espaços em branco do final de uma coluna (direita)

display(dataFrame.filter(ltrim(dataFrame.Cidade) == 'Brasilia'))

### # Utilizando ltrim e rtrim (remove espaços da esquerda e direita)

display(dataFrame.filter(ltrim(rtrim(dataFrame.Cidade)) == "Sao Jose dos Pinhais")) # Remove da esquerda e depois da direita.

display(dataFrame.filter(trim(dataFrame.Cidade) == "Sao Jose dos Pinhais")) # Remove de ambos os lados de uma só vez.


### # Resolvendo o 'problema' com palavras acentuadas


In [ ]:
from pyspark.sql.functions import regexp_replace

df_cidades = dataFrame.withColumn("CidadeSemAcento", regexp_replace(dataFrame.Cidade, "ã", "a")) # Altera 'ã' para 'a'
display(df_cidades)

## Alterar todos os acentos da língua portuguesa para não dar erro nas consultas ("normalizar" SEM ACENTO)

from pyspark.sql.functions import translate

# Precisa ser sempre na ordem abaixo:
Acentos = "áàãâéèêíìóòõôúùûçÁÀÃÂÉÈÊÍÌÓÒÕÔÚÙÛÇ"
SemAcentos = "aaaaeeeiioooouuucAAAAEEEIIOOOOUUUC"

# Aplicar a substituição utilizando a função translate
df_cidades = df_cidades.withColumn("CidadeSemAcento", translate(df_cidades.CidadeSemAcento, Acentos, SemAcentos))

display(df_cidades.filter(trim(df_cidades.CidadeSemAcento) == "Sao Jose dos Pinhais"))


## Removendo acento, espaço excessivo e deixando tudo maiúsculo

from pyspark.sql.functions import translate, upper, trim

Acentos = "áàãâéèêíìóòõôúùûçÁÀÃÂÉÈÊÍÌÓÒÕÔÚÙÛÇ"
SemAcentos = "aaaaeeeiioooouuucAAAAEEEIIOOOOUUUC"


df_cidades = df_cidades.withColumn("CidadeSemAcento", trim(upper(translate(df_cidades.Cidade, Acentos, SemAcentos))))

display(df_cidades)

### # Função LIKE


In [ ]:
# dbutils.fs.ls("/Volumes/workspace/default/arquivos-aula/Anac/V_OCORRENCIA_AMPLA.json")
# df = spark.read.json("/Volumes/workspace/default/arquivos-aula/Anac/V_OCORRENCIA_AMPLA.json")
# display(df)

# Filtro para um trecho específico

df = df.withColumn("Classificacao_da_Ocorrencia", regexp_replace(df.Classificacao_da_Ocorrência, "ê", "e"))

display(df)

resultado = df.filter(df.Classificacao_da_Ocorrencia.like("%Incidente")) # Utilizar o '.like'

# %Texto% --> Retorna tudo que está antes e após o texto na mesma célula
# Texto%  --> Retorna tudo que está depois do texto na mesma célula
# %Texto  --> Retorna tudo que está antes do texto na mesma célula

display(resultado)

### # Filtrando valores numéricos

In [ ]:
dbutils.fs.ls("/Volumes/workspace/default/arquivos-aula/Bike Store")

orders_df = spark.read.\
    format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-aula/Bike Store/orders.csv")

display(orders_df)

In [ ]:
### # Filtrando ordem cujo status é 1 

display(orders_df.filter(orders_df.order_status == 1))

### # Salvando em um novo DF simulando que o status '1' seja pedidos pendentes


pedidosPendentes = orders_df.filter(orders_df.order_status == 1)

display(pedidosPendentes)

### # Simulando pedidos de uma loja específica


display(pedidosPendentes.select("store_id").distinct().count()) # Descobrindo quantas lojas tem na coluna 'store_id' (Três lojas diferentes)

### # Selecionando a loja 1

loja1 = orders_df.filter(orders_df.store_id == 1)
display(loja1) # Retorna todos os dados em que a loja é a de 'ID' número 1 (store_id == 1)

### # Simulando pedidos diferentes de uma loja específica

# Retorna todos os dados das lojas 'menos' os da loja 1
# Um exemplo real é um cara que é Gerente das lojas 2 e 3 mas não é da loja 1, e pediu relatório das lojas dele (2 e 3 apenas).


fora1 = orders_df.filter(orders_df.store_id != 1)
display(fora1)

### Filtrando loja 3 com status 1 (filtro composto)

# Retorna todos os dados cujo status é 1 'E' a loja tem o id 2

statusUm = orders_df.filter((orders_df.order_status == 1) & (orders_df.store_id == 2))

display(statusUm)

### # Carregando novos dados da DBFS para consulta de preços

In [ ]:
dbutils.fs.ls("/Volumes/workspace/default/arquivos-aula/Bike Store/order_items.csv")

# Armazena os dados em um DataFrame e mostra ao final

order_items = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/arquivos-aula/Bike Store/order_items.csv")

display(order_items)

### # Filtrando por faixa de preços

In [ ]:
# Cria uma nova coluna chamada 'Opiniao' em que os produtos cujos preços sejam maiores ou igual a 300, as células serão preenchidas com 'O produto está caro' e as células cujos valores são menores que 300 serão preenchidas com 'O produto está barato'. E as 'NULL' serão preenchidas com 'Não avaliado'.


dfPrecos = order_items.withColumn(("Opiniao"), when(order_items.list_price >= 300, "O produto está caro").when(order_items.list_price <= 300, "O produto está barato")).fillna("Não avaliado") 

# .withColumn(("NomeDaNovaColuna"), when(nomeDoDataFrame.nomeDaColunaDeComparacao >= 300, "ResultadoImpressoNaCélula")

dfPrecos = dfPrecos.withColumnRenamed("order_id", "Id")\
                   .withColumnRenamed("item_id", "Item_Id")\
                   .withColumnRenamed("quantity", "Quantidade")\
                   .withColumnRenamed("product_id", "Produto_Id")\
                   .withColumnRenamed("list_price", "Preco")\
                   .withColumnRenamed("discount", "Desconto")

# Altera o nome das colunas para ficar visualmente mais bonito e padronizado em Pt-BR.

display(dfPrecos)

### # Criando uma nova coluna 'Estoque' e setando os indicadores 'Reposição URGENTE' em células com menos de 2 unidades, e 'Estoque Normal' em células com 2 ou mais unidades.

dfEstoque = dfPrecos.withColumn(("Estoque"), when(dfPrecos.Quantidade < 2, "Reposição URGENTE").when(dfPrecos.Quantidade >= 2, "Estoque Normal"))


display(dfEstoque)

### # Alterando o Produto_Id por um nome de produto 'fictício' somente para simular a criação de uma tabela e exportação da mesma em aquivo PDF ou outro formato externo.


In [ ]:
# Primeiro, selecionar a coluna Produto_Id para verificar com o 'distinct().count()' quantos IDs diferentes possuem, para eu saber quantos produtos terei que "criar", ou seja, quantos nomes de itens terei que informar. Depois, verificar os IDs diferentes apenas com o '.distinct()' para dai sim alterá-los.

display(dfEstoque.select("Produto_Id").distinct())

from pyspark.sql import SparkSession
from pyspark.sql import Row

nomes = ["Inha", "Guin", "AirJet", "AirKey", "AirLight", "AirLine", "AirLink", "AirMax", "AirNow", "AirOne", "AirPod", "AirStar", "AirWave", "AirX", "BrightGo", "BrightJet", "BrightMax", "BrightStar", "BrightX", "CleanEdge", "CleanFast", "CleanLab", "CleanLight", "CleanLine", "CleanMix", "CleanOne", "CleanPlus", "CleanUp", "EasyData", "EasyFast", "EasyFit", "EasyGear", "EasyGo", "EasyJet", "EasyKey", "EasyLab", "EasyLight", "EasyLine", "EasyMax", "EasyMix", "EasyMode", "EasyNow", "EasyOne", "EasyPlay", "EasyPlus", "EasyPod", "EasySet", "EasyStar", "EasyTop", "EasyTrack", "EasyUp", "EasyView", "EasyWave", "EasyX", "EasyZone", "EcoBox", "EcoClean", "EcoData", "EcoEdge", "EcoFast", "EcoFit", "EcoGear", "EcoGo", "EcoJet", "EcoKey", "EcoLab", "EcoLight", "EcoLine", "EcoLink", "EcoMax", "EcoMix", "EcoMode", "EcoNow", "EcoOne", "EcoPlay", "EcoPlus", "EcoPod", "EcoSet", "EcoStar", "EcoTop", "EcoTrack", "EcoUp", "EcoView", "EcoWave", "EcoX", "EcoZone", "FlexBox", "FlexData", "FlexEdge", "FlexFast", "FlexFit", "FlexGear", "FlexGo", "FlexJet", "FlexKey", "FlexLab", "FlexLight", "FlexLine", "FlexLink", "FlexMax", "FlexMix", "FlexMode", "FlexNow", "FlexOne", "FlexPlay", "FlexPlus", "FlexPod", "FlexSet", "FlexStar", "FlexTop", "FlexTrack", "FlexUp", "FlexView", "FlexWave", "FlexX", "FlexZone", "FreshBox", "FreshData", "FreshEdge", "FreshFast", "FreshFit", "FreshGear", "FreshGo", "FreshJet", "FreshKey", "FreshLab", "FreshLight", "FreshLine", "FreshLink", "FreshMax", "FreshMix", "FreshMode", "FreshNow", "FreshOne", "FreshPlay", "FreshPlus", "FreshPod", "FreshSet", "FreshStar", "FreshTop", "FreshTrack", "FreshUp", "FreshView", "FreshWave", "FreshX", "FreshZone", "GigaBox", "GigaData", "GigaEdge", "GigaFast", "GigaFit", "GigaGear", "GigaGo", "GigaJet", "GigaKey", "GigaLab", "GigaLight", "GigaLine", "GigaLink", "GigaMax", "GigaMix", "GigaMode", "GigaNow", "GigaOne", "GigaPlay", "GigaPlus", "GigaPod", "GigaSet", "GigaStar", "GigaTop", "GigaTrack", "GigaUp", "GigaView", "GigaWave", "GigaX", "GigaZone", "HyperBox", "HyperData", "HyperEdge", "HyperFast", "HyperFit", "HyperGear", "HyperGo", "HyperJet", "HyperKey", "HyperLab", "HyperLight", "HyperLine", "HyperLink", "HyperMax", "HyperMix", "HyperMode", "HyperNow", "HyperOne", "HyperPlay", "HyperPlus", "HyperPod", "HyperSet", "HyperStar", "HyperTop", "HyperTrack", "HyperUp", "HyperView", "HyperWave", "HyperX", "HyperZone", "LiteBox", "LiteData", "LiteEdge", "LiteFast", "LiteFit", "LiteGear", "LiteGo", "LiteJet", "LiteKey", "LiteLab", "LiteLight", "LiteLine", "LiteLink", "LiteMax", "LiteMix", "LiteMode", "LiteNow", "LiteOne", "LitePlay", "LitePlus", "LitePod", "LiteSet", "LiteStar", "LiteTop", "LiteTrack", "LiteUp", "LiteView", "LiteWave", "LiteX", "LiteZone", "MagicBox", "MagicData", "MagicEdge", "MagicFast", "MagicFit", "MagicGear", "MagicGo", "MagicJet", "MagicKey", "MagicLab", "MagicLight", "MagicLine", "MagicLink", "MagicMax", "MagicMix", "MagicMode", "MagicNow", "MagicOne", "MagicPlay", "MagicPlus", "MagicPod", "MagicSet", "MagicStar", "MagicTop", "MagicTrack", "MagicUp", "MagicView", "MagicWave", "MagicX", "MagicZone", "MaxBox", "MaxData", "MaxEdge", "MaxFast", "MaxFit", "MaxGear", "MaxGo", "MaxJet", "MaxKey", "MaxLab", "MaxLight", "MaxLine", "MaxLink", "MaxMix", "MaxMode", "MaxNow", "MaxOne", "MaxPlay", "MaxPlus", "MaxPod", "MaxSet", "MaxStar", "MaxTop", "MaxTrack", "MaxUp", "MaxView", "MaxWave", "MaxX", "MaxZone", "MegaBox", "MegaData", "MegaEdge", "MegaFast", "MegaFit", "MegaGear", "MegaGo", "MegaJet", "MegaKey", "MegaLab", "MegaLight", "MegaLine", "MegaLink", "MegaMax", "MegaMix", "MegaMode", "MegaNow", "MegaOne", "MegaPlay", "MegaPlus", "MegaPod", "MegaSet", "MegaStar", "MegaTop", "MegaTrack", "MegaUp", "MegaView", "MegaWave", "MegaX", "MegaZone", "MiniBox", "MiniData", "MiniEdge", "MiniFast", "MiniFit", "MiniGear", "MiniGo", "MiniJet", "MiniKey", "MiniLab", "MiniLight", "MiniLine", "MiniLink", "MiniMax", "MiniMix", "MiniMode", "MiniNow", "MiniOne", "MiniPlay", "MiniPlus", "MiniPod", "MiniSet", "MiniStar", "MiniTop", "MiniTrack", "MiniUp", "MiniView", "MiniWave", "MiniX", "MiniZone", "NeoBox", "NeoData", "NeoEdge", "NeoFast", "NeoFit", "NeoGear", "NeoGo", "NeoJet", "NeoKey", "NeoLab", "NeoLight", "NeoLine", "NeoLink", "NeoMax", "NeoMix", "NeoMode", "NeoNow", "NeoOne", "NeoPlay", "NeoPlus", "NeoPod", "NeoSet", "NeoStar", "NeoTop", "NeoTrack", "NeoUp", "NeoView", "NeoWave", "NeoX", "NeoZone", "NextBox", "NextData", "NextEdge", "NextFast", "NextFit", "NextGear", "NextGo", "NextJet", "NextKey", "NextLab", "NextLight", "NextLine", "NextLink", "NextMax", "NextMix", "NextMode", "NextNow", "NextOne", "NextPlay", "NextPlus", "NextPod", "NextSet", "NextStar", "NextTop", "NextTrack", "NextUp", "NextView", "NextWave", "NextX", "NextZone", "NovaBox", "NovaData", "NovaEdge", "NovaFast", "NovaFit", "NovaGear", "NovaGo", "NovaJet", "NovaKey", "NovaLab", "NovaLight", "NovaLine", "NovaLink", "NovaMax", "NovaMix", "NovaMode", "NovaNow", "NovaOne", "NovaPlay", "NovaPlus", "NovaPod", "NovaSet", "NovaStar", "NovaTop", "NovaTrack", "NovaUp", "NovaView", "NovaWave", "NovaX", "NovaZone", "OptiBox", "OptiData", "OptiEdge", "OptiFast", "OptiFit", "OptiGear", "OptiGo", "OptiJet", "OptiKey", "OptiLab", "OptiLight", "OptiLine", "OptiLink", "OptiMax", "OptiMix", "OptiMode", "OptiNow", "OptiOne", "OptiPlay", "OptiPlus", "OptiPod", "OptiSet", "OptiStar", "OptiTop", "OptiTrack", "OptiUp", "OptiView", "OptiWave", "OptiX", "OptiZone", "PowerBox", "PowerData", "PowerEdge", "PowerFast", "PowerFit", "PowerGear", "PowerGo", "PowerJet", "PowerKey", "PowerLab", "PowerLight", "PowerLine", "PowerLink", "PowerMax", "PowerMix", "PowerMode", "PowerNow", "PowerOne", "PowerPlay", "PowerPlus", "PowerPod", "PowerSet", "PowerStar", "PowerTop", "PowerTrack", "PowerUp", "PowerView", "PowerWave", "PowerX", "PowerZone", "PrimeBox", "PrimeData", "PrimeEdge", "PrimeFast", "PrimeFit", "PrimeGear", "PrimeGo", "PrimeJet", "PrimeKey", "PrimeLab", "PrimeLight", "PrimeLine", "PrimeLink", "PrimeMax", "PrimeMix", "PrimeMode", "PrimeNow", "PrimeOne", "PrimePlay", "PrimePlus", "PrimePod", "PrimeSet", "PrimeStar", "PrimeTop", "PrimeTrack", "PrimeUp", "PrimeView", "PrimeWave", "PrimeX", "PrimeZone", "QuickBox", "QuickData", "QuickEdge", "QuickFast", "QuickFit", "QuickGear", "QuickGo", "QuickJet", "QuickKey", "QuickLab", "QuickLight", "QuickLine", "QuickLink", "QuickMax", "QuickMix", "QuickMode", "QuickNow", "QuickOne", "QuickPlay", "QuickPlus", "QuickPod", "QuickSet", "QuickStar", "QuickTop", "QuickTrack", "QuickUp", "QuickView", "QuickWave", "QuickX", "QuickZone", "SmartBox", "SmartData", "SmartEdge", "SmartFast", "SmartFit", "SmartGear", "SmartGo", "SmartJet", "SmartKey", "SmartLab", "SmartLight", "SmartLine", "SmartLink", "SmartMax", "SmartMix", "SmartMode", "SmartNow", "SmartOne", "SmartPlay", "SmartPlus", "SmartPod", "SmartSet", "SmartStar", "SmartTop", "SmartTrack", "SmartUp", "SmartView", "SmartWave", "SmartX", "SmartZone", "SpeedBox", "SpeedData", "SpeedEdge", "SpeedFast", "SpeedFit", "SpeedGear", "SpeedGo", "SpeedJet", "SpeedKey", "SpeedLab", "SpeedLight", "SpeedLine", "SpeedLink", "SpeedMax", "SpeedMix", "SpeedMode", "SpeedNow", "SpeedOne", "SpeedPlay", "SpeedPlus", "SpeedPod", "SpeedSet", "SpeedStar", "SpeedTop", "SpeedTrack", "SpeedUp", "SpeedView", "SpeedWave", "SpeedX", "SpeedZone", "SuperBox", "SuperData", "SuperEdge", "SuperFast", "SuperFit", "SuperGear", "SuperGo", "SuperJet", "SuperKey", "SuperLab", "SuperLight", "SuperLine", "SuperLink", "SuperMax", "SuperMix", "SuperMode", "SuperNow", "SuperOne", "SuperPlay", "SuperPlus", "SuperPod", "SuperSet", "SuperStar", "SuperTop", "SuperTrack", "SuperUp", "SuperView", "SuperWave", "SuperX", "SuperZone", "TurboBox", "TurboData", "TurboEdge", "TurboFast", "TurboFit", "TurboGear", "TurboGo", "TurboJet", "TurboKey", "TurboLab", "TurboLight", "TurboLine", "TurboLink", "TurboMax", "TurboMix", "TurboMode", "TurboNow", "TurboOne", "TurboPlay", "TurboPlus", "TurboPod", "TurboSet", "TurboStar", "TurboTop", "TurboTrack", "TurboUp", "TurboView", "TurboWave", "TurboX", "TurboZone", "UltraBox", "UltraData", "UltraEdge", "UltraFast", "UltraFit", "UltraGear", "UltraGo", "UltraJet", "UltraKey", "UltraLab", "UltraLight", "UltraLine", "UltraLink", "UltraMax", "UltraMix", "UltraMode", "UltraNow", "UltraOne", "UltraPlay", "UltraPlus", "UltraPod", "UltraSet", "UltraStar", "UltraTop", "UltraTrack", "UltraUp", "UltraView", "UltraWave", "UltraX", "UltraZone", "ZoomBox", "ZoomData", "ZoomEdge", "ZoomFast", "ZoomFit", "ZoomGear", "ZoomGo", "ZoomJet", "ZoomKey", "ZoomLab", "ZoomLight", "ZoomLine", "ZoomLink", "ZoomMax", "ZoomMix", "ZoomMode", "ZoomNow", "ZoomOne", "ZoomPlay", "ZoomPlus", "ZoomPod", "ZoomSet", "ZoomStar", "ZoomTop", "ZoomTrack", "ZoomUp", "ZoomView", "ZoomWave", "ZoomX", "ZoomZone"]

display(nomes)

# Criar DataFrame de referência (ID, Nome)
df_produtos = spark.createDataFrame(
    [(i+1, nomes[i]) for i in range(len(nomes))],
    ["Produto_Id", "Nome_Produto"]
)

# Agora faz o join com o seu DF original
df_completo = dfEstoque.join(df_produtos, on="Produto_Id", how="left") # Cria um DF novo com os nomes de produtos


dfTabFinal = df_completo.select("Nome_Produto", "Estoque", "Preco", "Opiniao")

display(dfTabFinal)